# eFAST Interactive Runner

Select a site and season, then run eFAST processing and visualization.

**Setup:**
```bash
pip install ipywidgets matplotlib rasterio numpy
jupyter nbextension enable --py widgetsnbextension
```

In [22]:
from IPython.display import display
import ipywidgets as widgets
import importlib

from efast import interactive_helpers
importlib.reload(interactive_helpers)  # Reload to get latest changes

from efast.interactive_helpers import (
    setup_notebook, load_sites, 
    check_data_status, print_status_section, 
    create_download_button, create_processing_button,
    create_site_selection_widgets,
)

setup = setup_notebook()
test_data_dir = setup.test_data_dir


## 1. Site Selection

In [23]:
sites = load_sites('selected_sites.geojson')
site_dropdown, season_dropdown, info_output = create_site_selection_widgets(sites)

display(widgets.VBox([
    widgets.HTML("<h3>Select Site and Season</h3>"),
    widgets.HBox([site_dropdown, season_dropdown]),
    info_output
]))

## 2. Get Data

Download S2 and S3 data from CDSE, or use cached data if available.


In [24]:
from efast.interactive_helpers import download_data_if_needed

# Get selected site/season
sitename = site_dropdown.value
season_year = season_dropdown.value

if not sitename or not season_year:
    print("❌ Please select a site and season first")
else:
    print(f"📍 Cache location: {test_data_dir / sitename / season_year}")
    status = check_data_status(test_data_dir, sitename, season_year)
    print_status_section("DATA STATUS", status, fields=['s2_raw', 's3_raw'])
    
    # Automatically download if no cached data
    if status.s2_raw == 0 and status.s3_raw == 0:
        print("\n📥 No cached data found. Downloading...")
        download_data_if_needed(
            sites, sitename, season_year,
            test_data_dir, setup.run_efast_available
        )
        # Refresh status after download
        status = check_data_status(test_data_dir, sitename, season_year)
        print_status_section("DATA STATUS (after download)", status, fields=['s2_raw', 's3_raw'])
    else:
        print("\n✅ Data already cached")
        download_button, download_output = create_download_button(
            sites, site_dropdown, season_dropdown, test_data_dir, setup.run_efast_available
        )
        display(widgets.VBox([download_button, download_output]))

📍 Cache location: test_data/innsbruck/2024
📊 DATA STATUS
S2 Raw: ❌ 0 files
S3 Raw: ❌ 0 files

📥 No cached data found. Downloading...
❌ run_efast module not available
💡 Install dependencies: pip install creodias-finder pystac-client shapely tqdm
💡 Or set up the environment: conda env create -f environment.yml && pip install -e .
📊 DATA STATUS (after download)
S2 Raw: ❌ 0 files
S3 Raw: ❌ 0 files


## 3. Processing

Process S2 (extract and mask bands) and S3 (bin, composite, smooth, reproject) data. Uses cached results if available.


In [ ]:
sitename = site_dropdown.value
season_year = season_dropdown.value

if not sitename or not season_year:
    print("❌ Please select a site and season first")
else:
    status = check_data_status(test_data_dir, sitename, season_year)
    print_status_section("PROCESSING STATUS", status, fields=['s2_processed', 's3_reprojected'])
    
    process_button, process_output = create_processing_button(
        sites, site_dropdown, season_dropdown, test_data_dir, setup.efast_available
    )
    
    display(widgets.VBox([process_button, process_output]))


📊 PROCESSING STATUS
S2 Processed: ✅ 1 files
S3 Reprojected: ✅ 183 files
